# データ可視化のプロセス：ノーベル賞受賞者のデータ

Wikipediaからスクレイピングしてきたノーベル賞受賞者のデータを使って、データ可視化の6ステップをたどります。

In [ ]:
# ライブラリをロードする
import pandas as pd

## データの取得

スクレイピングはこの講義のスコープを超えるので、手元にすでにスクレイピング済みのデータがあると仮定します。

PythonによるWebスクレイピングに有用なライブラリとして、下記があります。

* [requests](https://requests-docs-ja.readthedocs.io/en/latest/): HTTP通信用のライブラリ
* [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/): XMLパース用のライブラリ
* [Scrapy](https://scrapy.org/): スクレイピング、クローリング用のフレームワーク

In [ ]:
# ファイルを読み込む



## データの